In [ ]:
import numpy
import pandas as pd
from pandas import read_csv
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv("train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227520 entries, 0 to 227519
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   tarih              227520 non-null  object 
 1   ürün               227520 non-null  object 
 2   ürün besin değeri  227520 non-null  int64  
 3   ürün kategorisi    227520 non-null  object 
 4   ürün fiyatı        227520 non-null  float64
 5   ürün üretim yeri   227520 non-null  object 
 6   market             227520 non-null  object 
 7   şehir              227520 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 13.9+ MB


In [3]:
df.tarih=df.tarih.apply(pd.to_datetime)
df['ts'] = df[['tarih']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)

In [4]:
le=LabelEncoder()
df["ürün"]=le.fit_transform(df["ürün"])
df["ürün üretim yeri"]=le.fit_transform(df["ürün üretim yeri"])
df["market"]=le.fit_transform(df["market"])
df["şehir"]=le.fit_transform(df["şehir"])
#

In [6]:
# split into train and test sets
x=df.loc[:,["ts","ürün","market","şehir","ürün üretim yeri"]]
y=df["ürün fiyatı"]
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.80,random_state=42)
rf = RandomForestRegressor(n_estimators=2000)
rf.fit(x_train,y_train)
sonuc=rf.predict(x_test)
r2 = r2_score(y_test, sonuc)
print(r2)

0.9940921610674596


In [7]:
df_2=pd.read_csv("testFeatures.csv")

df_2.tarih=df_2.tarih.apply(pd.to_datetime)
df_2['ts'] = df_2[['tarih']].apply(lambda x: x[0].timestamp(), axis=1)
df_2.tarih=df_2.tarih.apply(pd.to_datetime)
df_2["yıl"]=df_2.tarih.dt.year
df_2["ay"]=df_2.tarih.dt.month
df_2=df_2.drop("tarih",axis=1)


df_2["ürün"]=le.fit_transform(df_2["ürün"])
df_2["ürün üretim yeri"]=le.fit_transform(df_2["ürün üretim yeri"])
df_2["market"]=le.fit_transform(df_2["market"])
df_2["şehir"]=le.fit_transform(df_2["şehir"])
x_test2=df_2.loc[:,["ts","ürün","market","şehir","ürün üretim yeri"]]
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45504 entries, 0 to 45503
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 45504 non-null  int64  
 1   ürün               45504 non-null  int32  
 2   ürün besin değeri  45504 non-null  int64  
 3   ürün kategorisi    45504 non-null  object 
 4   ürün üretim yeri   45504 non-null  int32  
 5   market             45504 non-null  int32  
 6   şehir              45504 non-null  int32  
 7   ts                 45504 non-null  float64
 8   yıl                45504 non-null  int64  
 9   ay                 45504 non-null  int64  
dtypes: float64(1), int32(4), int64(4), object(1)
memory usage: 2.8+ MB


In [8]:
sonuc=rf.predict(x_test2)
sonuc_dp=pd.DataFrame(sonuc)
sonuc_dp=sonuc_dp.reset_index()
sonuc_dp=sonuc_dp.rename(columns={"index": "id",0:"ürün fiyatı"})
sonuc_dp.to_csv("sonuc.csv",index=False)